In [1]:
import sys
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import epitopepredict as ep
from epitopepredict import base, sequtils, analysis, plotting
from mhcflurry import Class1AffinityPredictor

This codes runs MHCflurry to obtain predictions for SARS-CoV-2 peptides. Results are filtered with netchop and then saved for further analysis

In [2]:
#get sequences from fasta file
# MN908947 the first Wuhan SARS-COV-2
fastafile="../SEQUENCES/sequences-Wuhan.fasta"
df = sequtils.fasta_to_dataframe(fastafile)

In [3]:
#keep only structural proteins M,N,E,S
df4=df.drop([0,1,4,5,7])


In [4]:
# select predictor
p = base.get_predictor('mhcflurry')

In [5]:
#select alleles HLA,HLB,HLC
predictor = Class1AffinityPredictor.load()
alleles=predictor.supported_alleles[8:88]


In [6]:
alleles.remove('HLA-B*07:01')

In [ ]:
predictor.predict_to_dataframe

In [19]:
#run predictions for the 4 structural proteins and 3 length
lengths=[8,9,10,11]
predictions4=[]
for ll in lengths:
    pred=p.predict_proteins(df4, length=ll, alleles=alleles)
    predictions4.append(pred)
#set into a dataframe
results = pd.concat(predictions4)

Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



predictions done for 4 sequences in 79 alleles
predictions done for 4 sequences in 79 alleles
predictions done for 4 sequences in 79 alleles
predictions done for 4 sequences in 79 alleles


In [7]:
# load results
#results=pd.read_csv("../DATA/results4_MHCflurry.csv")


In [20]:
# read list from NetChop
chop=pd.read_csv("../Data/cleavege-Wuhan-4.csv",sep=';')

In [21]:
# Create filter
filter=chop[chop["C"]=="S"]
filter=filter[["pos","Ident"]]
filter_key=filter[["pos","Ident"]].apply(tuple, axis=1)


In [22]:
idx = results[["pos","name"]].apply(lambda x: tuple(x) in list(filter_key.values), axis=1)

In [23]:
results_filter=results[idx].copy()

In [24]:
# compute global attachment
results_filter["K_tot"]=1./results_filter["prediction"]

/Users/szapperi/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
results_filter.to_csv("df_netMHCflurry_allHLA.csv")